In [1]:
import import_ipynb
import a_general_factor_test
import b_micro_factor_loader_library
import c_index_factor_loader
import rqdatac
import pandas as pd
rqdatac.init()
factors_selected = ['对PPI环比拉动_生产资料(全国:环比增长率:月)',
'avg_volume_60d',
'000300.XSHG',
 'volatility_90d',
 '银行间短期融资券收益率曲线（AA）:1年']

start_date = pd.to_datetime('2018-01-01')
end_date = pd.to_datetime('2020-12-31')
instrument_type = 'Convertible'
remaining_time_to_mature ='短期 (1-3年)' #labels = ['短期 (1-3年)', '中期 (3-5年)', '长期 (5+年)']
rolling_window = 90


/opt/anaconda3/envs/rqsdk/lib/python3.9/site-packages/rqdatac/client.py:257: UserWarning: Your account will be expired after  24 days. Please call us at 0755-22676337 to upgrade or purchase or renew your contract.
  warnings.warn("Your account will be expired after  {} days. "


In [2]:
bonds_poll = a_general_factor_test.get_bonds_poll(start_date=start_date,end_date=end_date,instrument_type=instrument_type)
dependent = a_general_factor_test.get_return_rate(all_instruments=bonds_poll,start_date=start_date,end_date=end_date)
print(dependent)

testmodel2
可转债数量: 449
                          return_30  daily_return
order_book_id date                               
110030.XSHG   2018-01-03   0.025033      0.000569
              2018-01-04   0.029356      0.001328
              2018-01-05  -0.016737      0.035417
              2018-01-08  -0.024964      0.003841
              2018-01-09  -0.019051     -0.005284
...                             ...           ...
132022.XSHG   2020-12-16  -0.008317      0.006979
              2020-12-17  -0.005941      0.000000
              2020-12-18  -0.005446      0.000000
              2020-12-21  -0.009755      0.004851
              2020-12-22  -0.005760     -0.007784

[129384 rows x 2 columns]


In [3]:
def econ_loader(factors_selected,start_date,end_date):
    assert isinstance(factors_selected,str),'输入应该是一个挑选好的econ factor的string'
    r_df = a_general_factor_test.econ_get_factor_value_forward_fill(factors_selected,start_date=start_date,end_date=end_date)
    return r_df

print(econ_loader(factors_selected[0],start_date=start_date,end_date=end_date))
    

testing: 对PPI环比拉动_生产资料(全国:环比增长率:月)
            value
date             
2018-01-10   0.71
2018-01-11   0.71
2018-01-12   0.71
2018-01-13   0.71
2018-01-14   0.71
...           ...
2020-12-27   0.50
2020-12-28   0.50
2020-12-29   0.50
2020-12-30   0.50
2020-12-31   0.50

[1087 rows x 1 columns]


In [4]:
def micro_loader(factor,start_date,end_date,bonds_poll):
    assert isinstance(factor,str),f'输入应该是一个挑选好的micro factor的string,现在类型是{type(factor)}'
    r_df = b_micro_factor_loader_library.generate_micro_factors(all_instruments=bonds_poll,start_date=start_date,end_date=end_date,factor_name=factor)
    return r_df
print(micro_loader(factor=factors_selected[1],start_date=start_date,end_date=end_date,bonds_poll=bonds_poll))

开始为因子 'avg_volume_60d' 获取基础量价数据...
数据获取完毕，开始计算因子...
正在计算流动性因子: avg_volume_60d...
因子 avg_volume_60d 计算完成。
       order_book_id       date           beta
0        110030.XSHG 2018-01-03   94290.000000
1        110030.XSHG 2018-01-04   91780.000000
2        110030.XSHG 2018-01-05  187905.000000
3        110030.XSHG 2018-01-08  212894.000000
4        110030.XSHG 2018-01-09  218633.333333
...              ...        ...            ...
133910   132022.XSHG 2020-12-25   22408.000000
133911   132022.XSHG 2020-12-28   22533.833333
133912   132022.XSHG 2020-12-29   22513.666667
133913   132022.XSHG 2020-12-30   21822.166667
133914   132022.XSHG 2020-12-31   21675.833333

[133915 rows x 3 columns]


In [7]:
def index_loader(factor,start_date,end_date):
    assert isinstance(factor,str),'The factor should be a string'
    r_df = c_index_factor_loader.get_profolio_factor(factor,start_date,end_date)
    return r_df
print(index_loader(factors_selected[2],start_date=start_date,end_date=end_date))


               value
date                
2018-01-03  0.005870
2018-01-04  0.004237
2018-01-05  0.002407
2018-01-08  0.005173
2018-01-09  0.007004
...              ...
2020-12-25  0.008400
2020-12-28  0.004443
2020-12-29 -0.004241
2020-12-30  0.014034
2020-12-31  0.019082

[729 rows x 1 columns]


In [10]:
def prepare_muti_factor_data(bonds_poll,factors_selected,start_date,end_date):
    print(factors_selected)
    avg_volume_60d = micro_loader('avg_volume_60d',start_date=start_date,end_date=end_date,bonds_poll=bonds_poll)
    volatility_90d = micro_loader('volatility_90d',start_date=start_date,end_date=end_date,bonds_poll=bonds_poll)
    ppi = econ_loader('对PPI环比拉动_生产资料(全国:环比增长率:月)',start_date,end_date)
    AA = econ_loader('银行间短期融资券收益率曲线（AA）:1年',start_date,end_date)
    XSHG = index_loader('000300.XSHG',start_date=start_date,end_date=end_date)
df = prepare_muti_factor_data(bonds_poll=bonds_poll,factors_selected= factors_selected,start_date=start_date,end_date=end_date)

['对PPI环比拉动_生产资料(全国:环比增长率:月)', 'avg_volume_60d', '000300.XSHG', 'volatility_90d', '银行间短期融资券收益率曲线（AA）:1年']
开始为因子 'avg_volume_60d' 获取基础量价数据...
数据获取完毕，开始计算因子...
正在计算流动性因子: avg_volume_60d...
因子 avg_volume_60d 计算完成。
开始为因子 'volatility_90d' 获取基础量价数据...
数据获取完毕，开始计算因子...
正在计算波动率因子: volatility_90d...
因子 volatility_90d 计算完成。
testing: 对PPI环比拉动_生产资料(全国:环比增长率:月)
testing: 银行间短期融资券收益率曲线（AA）:1年
